In [1]:
import sys
sys.path.append('../lib')
sys.path.append('../src')
sys.path.append('../src/queries')
sys.path.append('../src/clustering')
sys.path.append('../src/visualization_lib')
sys.path.append('../data')
sys.path.append('../')

%matplotlib inline


from lib import csv_reader as reader
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math
import json

from lib import year_classifier as year_classifier

from jinja2 import Environment, FileSystemLoader

In [3]:
from itertools import chain, combinations
def frequency_matrix(start, end):
    dimensions =    110 
    matrix = np.zeros(shape=(dimensions,dimensions), dtype=np.int)

    clusters = df.query('mid_year >= {0} & mid_year <= {1}'.format(start, end))

    cluster_list = clusters.groupby('picture_id')['cluster_id'].apply(list)
    cluster_list = np.array(cluster_list)
    for transaction in cluster_list:
        for z in chain.from_iterable(combinations(transaction, r) for r in range(len(transaction)+1)):
            if(len(z) == 2):
                i, j = z
                matrix[i, j] += 1
                matrix[j, i] += 1
    return matrix


#load Data!!!
import db

my_db = db.Db()
df = my_db.final_cluster_nation

time_ranges = [[1789,1847], [1848,1874], [1875,1914], [1789,1914]]

# nicht das gleiche ergebnis, da zeitraum kleiner
#df['mid_year'].max() = 1918


time_range_matrices = []

for x in time_ranges:
    mat = frequency_matrix(x[0], x[1])
    #time_range_matrices.append(pd.DataFrame(mat))
    time_range_matrices.append(mat)
    
time_range_matrices

Db initialized, data is now accessible. It took  1.38s reading all files


[array([[ 0,  2,  5, ...,  1,  2, 78],
        [ 2,  0,  0, ..., 28,  0,  0],
        [ 5,  0,  0, ..., 58,  0,  2],
        ..., 
        [ 1, 28, 58, ...,  0,  7,  4],
        [ 2,  0,  0, ...,  7,  0,  3],
        [78,  0,  2, ...,  4,  3,  0]]), array([[ 0,  1,  2, ...,  2,  2, 22],
        [ 1,  0,  0, ..., 86,  3,  1],
        [ 2,  0,  0, ..., 42,  0,  2],
        ..., 
        [ 2, 86, 42, ...,  0, 14, 10],
        [ 2,  3,  0, ..., 14,  0,  1],
        [22,  1,  2, ..., 10,  1,  0]]), array([[ 0,  0,  5, ...,  3,  1, 20],
        [ 0,  0,  2, ..., 31,  0,  0],
        [ 5,  2,  0, ..., 52,  2,  4],
        ..., 
        [ 3, 31, 52, ...,  0, 15,  7],
        [ 1,  0,  2, ..., 15,  0,  5],
        [20,  0,  4, ...,  7,  5,  0]]), array([[  0,   3,  12, ...,   6,   5, 120],
        [  3,   0,   2, ..., 145,   3,   1],
        [ 12,   2,   0, ..., 152,   2,   8],
        ..., 
        [  6, 145, 152, ...,   0,  36,  21],
        [  5,   3,   2, ...,  36,   0,   9],
        [120, 

In [6]:
mat = time_range_matrices[3]
df_mat = pd.DataFrame(mat)
df_mat

,0,1,2,3,4,5,6,7,8,9,...,100,101,102,103,104,105,106,107,108,109
0,0,3,12,23,44,19,28,12,2,6,...,82,0,2,4,6,44,5,6,5,120
1,3,0,2,1,4,43,1,4,95,1,...,0,44,3,95,114,8,1,145,3,1
2,12,2,0,6,4,172,5,14,14,1,...,5,1,2,3,2,12,2,152,2,8
3,23,1,6,0,9,131,30,24,27,305,...,1,0,34,7,107,16,4,22,13,40
4,44,4,4,9,0,9,9,4,1,2,...,41,0,4,0,3,23,7,19,5,108
5,19,43,172,131,9,0,55,78,28,38,...,5,8,5,37,88,13,5,7,12,18
6,28,1,5,30,9,55,0,16,1,7,...,2,0,7,9,1,23,4,23,5,7
7,12,4,14,24,4,78,16,0,5,11,...,26,1,5,2,17,18,3,33,1,18
8,2,95,14,27,1,28,1,5,0,18,...,2,18,7,42,72,1,8,107,10,6
9,6,1,1,305,2,38,7,11,18,0,...,0,2,19,3,69,9,1,12,5,18


4

In [95]:
# matrix to Nodes and links

jsondata = json.loads('{"nodes": [], "links": []}')

#"nodes": [
#    {"id": "Myriel", "group": 1},

valueMin = 80

topic_group_women=[3,9,45,97]
topic_group_men=[79, 1,43, 81, 55]
topic_group_war=[62,91,53,105,21]
topic_group_religion=[52, 85, 96, 15, 36, 67, 82, 88]
topic_group_landscape=[0, 27, 10, 51, 13, 17, 41, 60 ,63, 109, 77]
topic_group_portraits=[101, 103, 69, 92, 56, 69]

import json
def append_cluster_name(df, column='cluster_name'):
    # TODO!!!
    with open('../data/topics_benannt.txt') as topic_word_json:
        topic_word_dic = json.load(topic_word_json)
    
    df['cluster_name'] = df['cluster_id'].apply(lambda x: topic_word_dic[str(x)])
    return df

nodes = pd.DataFrame(range(0,110), columns=['cluster_id'])

nodes['group'] = nodes.cluster_id.map(lambda x: \
     1 if x in topic_group_women else \
     2 if x in topic_group_men else \
     3 if x in topic_group_war else \
     4 if x in topic_group_religion else \
     5 if x in topic_group_landscape else \
     6 if x in topic_group_portraits else 0 )

nodes = append_cluster_name(nodes)
json_nodes = json.loads(nodes.to_json(orient='records'))
jsondata['nodes'] = json_nodes

# links

#"links": [
#    {"source": "Napoleon", "target": "Myriel", "value": 1},



for x in range(110):
    for y in xrange(x+1,110):
        val = df_mat[x][y]
        if val >valueMin:
            jsondata["links"].append(json.loads('{'+'"source": "{0}", "target": "{1}", "value": {2}'.format(x,y,val)+'}'))

with open('graph/graph_data.js', 'w') as outfile:
    outfile.write("graph = ")
    json.dump(jsondata, outfile, sort_keys=True, indent=4)
    #json.dump(jsondata, outfile)    
print 'saved' + ' /graph/graph_data.js' + ' with valueMin: ' + str(valueMin) + '.'
jsondata

{u'links': [{u'source': u'0', u'target': u'10', u'value': 129},
  {u'source': u'0', u'target': u'13', u'value': 89},
  {u'source': u'0', u'target': u'27', u'value': 164},
  {u'source': u'0', u'target': u'46', u'value': 99},
  {u'source': u'0', u'target': u'51', u'value': 125},
  {u'source': u'0', u'target': u'60', u'value': 145},
  {u'source': u'0', u'target': u'70', u'value': 110},
  {u'source': u'0', u'target': u'100', u'value': 82},
  {u'source': u'0', u'target': u'109', u'value': 120},
  {u'source': u'1', u'target': u'8', u'value': 95},
  {u'source': u'1', u'target': u'43', u'value': 242},
  {u'source': u'1', u'target': u'69', u'value': 299},
  {u'source': u'1', u'target': u'79', u'value': 104},
  {u'source': u'1', u'target': u'103', u'value': 95},
  {u'source': u'1', u'target': u'104', u'value': 114},
  {u'source': u'1', u'target': u'107', u'value': 145},
  {u'source': u'2', u'target': u'5', u'value': 172},
  {u'source': u'2', u'target': u'12', u'value': 112},
  {u'source': u'2', 